In [1]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [20]:
zu = np.inf
zl = 0

In [43]:
A = np.array([
    [3, 1, 2, 4, 1],
    [2, 3, 1, 4, 2]
])
b = np.array([6, 5])

In [44]:
lambd = np.zeros(A.shape[0])
lambd

array([0., 0.])

In [45]:
Ct = [sum([(1 - lambd[r]) * A[r][c] for r in range(b.shape[0])]) 
      for c in range(A.shape[-1])]
Ct

[5.0, 4.0, 3.0, 8.0, 3.0]

In [46]:
x = [0 if ct >= 0 else 1 for ct in Ct]
x

[0, 0, 0, 0, 0]

In [47]:
C_t = (1 - lambd) @ A
x = np.where(C_t < 0, 1, 0)
C_t

array([5., 4., 3., 8., 3.])

In [48]:
L = C_t @ x + lambd @ b
L

0.0

In [49]:
zl = max(zl, L)
zl

0

In [50]:
g = b - A @ x
g

array([6, 5])

In [165]:
A = np.array([
    [1, 2, 4],
    [3, 1, 4],
    [2, 2, 2]
])
b = np.array([2, 5, 1])

In [187]:
def subgrad_opt(
        A, b, z_ub, z_lb, lambd, f=2, k=15, eps=0.005, omega=500):
    # TODO: z_ub cannot be inf here
    unchanged = 0
    t = 0
    lambd_best = lambd
    x_best = None
    while (z_ub > z_lb):
        c = (1 - lambd) @ A
        # print("c: ", c)
        x = np.where(c < 0, 1, 0)
        # print(x)
        L = c @ x + lambd @ b
        # print("L: ", L)
        g = b - A @ x
        if L > z_lb:
            z_lb = L
            lambd_best = lambd
            x_best = x
            unchanged = 0
            # print("L new: ", L)
        else:
            unchanged += 1
        if unchanged == k:
            unchanged = 0
            f /= 2
        sigma = f * (z_ub - z_lb) / np.linalg.norm(g) ** 2
        lambd = np.maximum(np.zeros_like(lambd), lambd + sigma * g)
        # lambd = np.minimum(1, lambd)
        # print("lambd: ", lambd)
        t += 1
        if f < eps or t > omega:
            print(f, t, g)
            break
    
    return lambd_best, z_lb, x_best, f

In [188]:
subgrad_opt(A, b, 100, 0, np.zeros(A.shape[0]))

0.00390625 217 [-3 -2 -3]


(array([0.       , 2.5099687, 0.       ]),
 10.980062602068603,
 array([1, 0, 1]),
 0.00390625)